In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 입력 데이터
sentences = ["i like dog", "i love coffee", "i hate milk"]

# vocab 생성
vocab = list(set(' '.join(sentences).split()))
print('vocab : ', vocab)

vocab :  ['hate', 'love', 'milk', 'dog', 'like', 'coffee', 'i']


In [7]:
# 각 단어에 고유 인덱스 부여
word2index = {tkn: i for i, tkn in enumerate(vocab)}
print('word2index : ', word2index)

word2index :  {'hate': 0, 'love': 1, 'milk': 2, 'dog': 3, 'like': 4, 'coffee': 5, 'i': 6}


In [8]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
# word2index의 반대 개념
index2word = {v: k for k, v in word2index.items()}
print('index2word : ', index2word)

index2word :  {0: 'hate', 1: 'love', 2: 'milk', 3: 'dog', 4: 'like', 5: 'coffee', 6: 'i'}


In [9]:
def make_batch(sentences, word2index, input_size):
    """
    배치를 만드는 함수 이번 예제에서는 

    파라미터 
    ---
    sentences : list
        각 단어가 들어있는 list
    word2index : dict
        단어 별 고유 인덱스가 매핑 되어 있는 dict
    input_size : int
        vocab의 사이즈 

    returns 
    ---
    input_batch  : list
        입력 데이터, 문장중 가장 마지막 단어를 제외한 모든 단어
    target_batch : list
        target 데이터, 문장의 가장 마지막 단어
    """
    input_batch = []
    target_batch = []

    for sen in sentences:
        # 단어를 공백을 기준으로 split
        word = sen.split()
        # 문장 중 가장 마지막 단어를 제외한 모든 단어를 input 으로
        # word2index를 이용하여 input의 각 단어를 고유 인덱스 매핑
        input = [word2index[n] for n in word[:-1]]

        # 문장 중 가장 마지막 단어를 target으로
        # word2index를 이용하여 target 단어를 고유 인덱스 매핑
        target = word2index[word[-1]]

        input_batch.append(np.eye(input_size)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [40]:
class Net(nn.Module):
    """
    간단한 rnn 함수
    2개의 단어를 input으로 받아서 다음에 올 단어를 예측하는 모델
    """

    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()

        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size,
                          batch_first=True)

        # lstm 사용
        #self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)  # 출력은 단어 집합의 크기로 지정

    def forward(self, x):
        # rnn모델의 출력 값은 입력 데이터에 대한 출력과
        # 다음 rnn셀로 보낼 hidden state

        output, _ = self.rnn(x)
        # lstm사용
        #output, (hidden,cell) = self.lstm(x)

        # input 단어가 2개이기 때문에 2개 단어에 대한 출력 벡터가 생성되는데
        # 2번째 단어(마지막으로 입력된 단어)의 출력 벡터만 사용
        output = output[:, -1, :]

        output = self.linear(output)

        return output

In [41]:
input_size = 7
hidden_size = 8
output_size = 7

model = Net(input_size, hidden_size, output_size)

In [42]:
input_batch, target_batch = make_batch(sentences, word2index, input_size)
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
# Training
for epoch in range(5000):
    optimizer.zero_grad()

    # input_batch : [batch_size, n_step, n_class]
    output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)

    loss = criterion(output, target_batch)

    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.060574
Epoch: 2000 cost = 0.016163
Epoch: 3000 cost = 0.006953
Epoch: 4000 cost = 0.003536
Epoch: 5000 cost = 0.001946
